In [15]:
import glob, os
import pandas as pd

file_list = glob.glob('./data/*.csv') 
df = pd.DataFrame()


for f in file_list:
    df_tmp = pd.read_csv(f)
    i = f.find('cy')
    df_tmp['year'] = f[i+2:i+6]
    i = f.find('\\')
    df_tmp['sex'] = f[i+1]
    df = pd.concat([df, df_tmp],ignore_index=True)

In [4]:
df['Position'] = df.Position.str.replace('=','')

In [5]:
df.Position.str.contains('=').sum()

0

In [14]:
dup_val = df[df.duplicated(['Given Name','year'],keep=False)].groupby(['Given Name','year']).Amount.sum()


In [21]:
dup_val.shape[0]

8711

In [18]:
dup_val.head()

Given Name  year
(NOT        1968     8
AARON       1963     8
            1966     7
            1968    21
            1969    10
Name: Amount, dtype: int64

In [22]:
df.shape[0]

210939

In [23]:
df.drop_duplicates(['Given Name','year'],keep=False)

,Given Name,Amount,Position,year,sex
1,HELEN,209,2,1944,f
2,JUDITH,189,3,1944,f
3,PATRICIA,186,4,1944,f
4,PAMELA,160,5,1944,f
5,ELIZABETH,153,6,1944,f
6,JENNIFER,149,7,1944,f
8,LORRAINE,143,9,1944,f
9,CHRISTINE,135,10,1944,f
11,JANICE,107,=12,1944,f
13,LYNETTE,106,14,1944,f


In [13]:
df = df.sort_values(['year','Position']).reset_index(drop=True)

In [13]:
df.to_csv("all_names.csv")

you need to merge rows if name is the same!!!

In [12]:
df.head(100)

,Given Name,Amount,Position,year,sex
0,MARGARET,341,1,1944,f
1,JOHN,477,1,1944,m
2,CHRISTINE,135,10,1944,f
3,GEOFFREY,143,10,1944,m
4,ELLEN,11,103,1944,f
5,ERICA,11,103,1944,f
6,GAIL,11,103,1944,f
7,GWENDA,11,103,1944,f
8,JULIA,11,103,1944,f
9,MERILYN,11,103,1944,f
